## Introdução

Começaremos com um breve tutorial sobre regressão, recursos polinomiais e regularização com base em um conjunto de dados muito simples e esparso que contém uma coluna de dados 'x' e dados ruidosos associados a 'y'. O arquivo de dados é chamado de 'X_Y_Sinusoid_Data.csv'.

## Questão 1

* Importar os dados.

* Também gere aproximadamente 100 pontos de dados x igualmente espaçados no intervalo de 0 a 1. Usando esses pontos, calcule os dados y que representam a "verdade básica" (a função real) da equação: $y = sin(2\ pix)$

* Plote os dados esparsos (`x` vs `y`) e os dados calculados ("reais").

In [ ]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

In [ ]:
import pandas as pd
import numpy as np


data = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-ML240EN-SkillsNetwork/labs/data/X_Y_Sinusoid_Data.csv")
data.head()

X_real = np.linspace(0, 1.0, 100)
Y_real = np.sin(2 * np.pi * X_real)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [ ]:
sns.set_style('white')
sns.set_context('talk')
sns.set_palette('dark')

# Plot of the noisy (sparse)
ax = data.set_index('x')['y'].plot(ls='', marker='o', label='data')
ax.plot(X_real, Y_real, ls='--', marker='', label='real function')

ax.legend()
ax.set(xlabel='x data', ylabel='y data');


## Questão 2

* Usando a classe `PolynomialFeatures` da biblioteca de pré-processamento do Scikit-learn, crie recursos polinomiais de 20ª ordem.
* Ajuste esses dados usando regressão linear.
* Plote o valor previsto resultante em comparação com os dados calculados.

Observe que `PolynomialFeatures` requer um dataframe (com uma coluna, não uma série) ou uma matriz 2D de dimensão (`X`, 1), onde `X` é o comprimento.

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression

# Configurei os recursos polinomiais
degree = 20
pf = PolynomialFeatures(degree)
lr = LinearRegression()

# Extrai os dados X e Y do dataframe
X_data = data[['x']]
Y_data = data['y']

# Criei os recursos e ajuste o modelo
X_poly = pf.fit_transform(X_data)
lr = lr.fit(X_poly, Y_data)
Y_pred = lr.predict(X_poly)

# plotei os resultados 
plt.plot(X_data, Y_data, marker='o', ls='', label='data', alpha=1)
plt.plot(X_real, Y_real, ls='--', label='real function')
plt.plot(X_data, Y_pred, marker='^', alpha=.5, label='predictions w/ polynomial features')
plt.legend()
ax = plt.gca()
ax.set(xlabel='x data', ylabel='y data');


## Questão 3

* Execute a regressão usando os dados com características polinomiais usando regressão ridge ($\alpha$=0.001) e regressão lasso ($\alpha$=0.0001).
* Plote os resultados, como foi feito na Questão 1.
* Trace também a magnitude dos coeficientes obtidos nessas regressões e compare-os com os obtidos na regressão linear da questão anterior. Os coeficientes de regressão linear provavelmente precisarão de um gráfico separado (ou de seu próprio eixo y) devido à sua grande magnitude.

O que a magnitude comparativamente grande dos dados diz sobre o papel da regularização?

In [ ]:
# silenciei os avisos sobre a execucao do treinamento
import warnings
warnings.filterwarnings('ignore', module='sklearn')

from sklearn.linear_model import Ridge, Lasso

# O modelo de regressão do cume

rr = Ridge(alpha=0.001)
rr = rr.fit(X_poly, Y_data)
Y_pred_rr = rr.predict(X_poly)

# O modelo de regressão laço
lassor = Lasso(alpha=0.0001)
lassor = lassor.fit(X_poly, Y_data)
Y_pred_lr = lassor.predict(X_poly)

# O gráfico dos valores previstos
plt.plot(X_data, Y_data, marker='o', ls='', label='data')
plt.plot(X_real, Y_real, ls='--', label='real function')
plt.plot(X_data, Y_pred, label='linear regression', marker='^', alpha=.5)
plt.plot(X_data, Y_pred_rr, label='ridge regression', marker='^', alpha=.5)
plt.plot(X_data, Y_pred_lr, label='lasso regression', marker='^', alpha=.5)

plt.legend()

ax = plt.gca()
ax.set(xlabel='x data', ylabel='y data');

In [ ]:
# valor absoluto dos coeficientes para cada modelo

coefficients = pd.DataFrame()
coefficients['linear regression'] = lr.coef_.ravel()
coefficients['ridge regression'] = rr.coef_.ravel()
coefficients['lasso regression'] = lassor.coef_.ravel()
coefficients = coefficients.applymap(abs)

coefficients.describe()  #diferença na escala entre regressão não regularizada versus regularizada

In [ ]:
colors = sns.color_palette()

# Configurei os eixos y duplos
ax1 = plt.axes()
ax2 = ax1.twinx()

# Plotei os dados de regressão linear
ax1.plot(lr.coef_.ravel(), 
         color=colors[0], marker='o', label='linear regression')

# Plotei os conjuntos de dados de regularização
ax2.plot(rr.coef_.ravel(), 
         color=colors[1], marker='o', label='ridge regression')

ax2.plot(lassor.coef_.ravel(), 
         color=colors[2], marker='o', label='lasso regression')

# Personalizei escalas de eixos
ax1.set_ylim(-2e14, 2e14)
ax2.set_ylim(-25, 25)

# combinei as legendas
h1, l1 = ax1.get_legend_handles_labels()
h2, l2 = ax2.get_legend_handles_labels()
ax1.legend(h1+h2, l1+l2)

ax1.set(xlabel='coefficients',ylabel='linear regression')
ax2.set(ylabel='ridge and lasso regression')

ax1.set_xticks(range(len(lr.coef_)));